## 1-Axis shading PVSC2018

Example demonstrating Radiance gencumulativesky for 1-axis tracking.

Updated with v0.2.2 bifacial_radiance which corrects some 1-axis tracking errors.

#### Prerequisites (Step 0):
This software requires the previous installation of RADIANCE from https://github.com/NREL/Radiance/releases.

Make sure you add radiance to the system PATH so Python can interact with the radiance program

If you are on a PC you should also copy the Jaloxa radwinexe-5.0.a.8-win64.zip executables into `program files/radiance/bin`: http://www.jaloxa.eu/resources/radiance/radwinexe.shtml

#### STEP 1: Install and import bifacial_radiance

 - clone the bifacial_radiance repo to your local directory
 - navigate to the \bifacial_radiance directory which contains setup
 - run `pip install -e .  `  ( the period . is required, the -e flag is optional and installs in development mode where changes to the bifacial_radiance.py files are immediately incorporated into the module if you re-start the python kernel)

#### STEP 2: Move gencumulativesky.exe
Copy gencumulativesky.exe from the repo's `/bifacial_radiance/data/` directory and copy into your Radiance install directory.
This is typically found in `/program files/radiance/bin/`.  

#### STEP 3: Create a local Radiance directory for storing the scene files created
Keep scene geometry files separate from the bifacial_radiance directory.  Create a local directory somewhere that will be referenced in the next step.

#### STEP 4: Reboot the computer
This makes sure the PATH is updated




In [1]:
from __future__ import division
import math
import numpy as np
import pandas as pd
import os

testfolder = os.path.abspath(r'..\bifacial_radiance\TEMP')  
print ("Your simulation will be stored in %s" % testfolder)
# tracker geometry options:
module_height = 2  # module portrait dimension in meters
gcr = 0.35   # ground cover ratio,  = module_height / pitch
albedo = 'concrete'     # ground albedo
hub_height = module_height*0.75   # H = 0.75 
limit_angle = 45 # tracker rotation limit angle

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


In [5]:
import bifacial_radiance
demo = RadianceObj(path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.



ImportError: cannot import name 'AnalysisObj' from 'bifacial_radiance' (C:\ProgramData\Anaconda3\lib\site-packages\bifacial_radiance\__init__.py)

In [ ]:
#epwfile = demo.getEPW(37.5,-77.6) #Pull TMY data for any global lat/lon. In this case, Richmond, VA
epwfile = demo.getEPW(35.1,-106.7) #Pull TMY data for any global lat/lon. In this case, ABQ
metdata = demo.readEPW(epwfile) # read in the weather data
# sunny day 6/24/1972 (index 4180 - 4195)


In [ ]:
# create a dataframe to make filtering easier

temp = pd.DataFrame({'datetime':metdata.datetime,'ghi':metdata.ghi,'dhi':metdata.dhi,'dni':metdata.dni})
df = temp[(temp['datetime'] < '1972-06-24 21:00:00-07:00') & (temp['datetime'] > '1972-06-24 5:00:00-07:00')]
metdata.datetime = df.datetime.tolist()
metdata.ghi = df.ghi.tolist()
metdata.dhi = df.dhi.tolist()
metdata.dni = df.dni.tolist()
df['numIndex'] = np.linspace(0,df.__len__()-1,df.__len__(),dtype=int)

df.index = df.datetime

In [ ]:
trackingdata = metdata._getTrackingAngles()
df = df.join(trackingdata)
df.index = df['numIndex']

In [ ]:

    
def linePtsMakeDict(linePtsDict):
    a = linePtsDict
    linepts = linePtsMake3D(a['xstart'],a['ystart'],a['zstart'],
                            a['xinc'], a['yinc'], a['zinc'],
                            a['Nx'],a['Ny'],a['Nz'],a['orient'])
    return linepts
def linePtsMake3D(xstart,ystart,zstart,xinc,yinc,zinc,Nx,Ny,Nz,orient):
        #linePtsMake(xpos,ypos,zstart,zend,Nx,Ny,Nz,dir)
        #create linepts text input with variable x,y,z. 
        #If you don't want to iterate over a variable, inc = 0, N = 1.
        # Add new line to simultaneously scan x and z.
        
        #now create our own matrix - 3D nested Z,Y,Z
        linepts = ""
        for iz in range(0,Nz):
            zpos = zstart+iz*zinc
            for iy in range(0,Ny):
                ypos = ystart+iy*yinc
                for ix in range(0,Nx):
                    xpos = xstart+ix*xinc
                    zpos = zstart+ix*zinc  # new line to simultaneously scan x and z. Nz is still 1
                    linepts = linepts + str(xpos) + ' ' + str(ypos) + ' '+str(zpos) + ' ' + orient + " \r"
        return(linepts)
    
def saveResults(data, reardata = None, savefile = None):
    ''' 
    saveResults - function to save output from irrPlotNew 
    If rearvals is passed in, back ratio is saved
    PVSC special output to include rear z and rear material type

    Returns: savefile       
    '''
    import os
    if savefile is None:
        savefile = data['title'] + '.csv'
    # make dataframe from results
    data_sub = {key:data[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

    if reardata is not None:
        rearMat = reardata['mattype']
        data_sub['rearMat'] = rearMat
        rearZ = reardata['z']
        data_sub['rearZ'] = rearZ
        Wm2Front = data_sub.pop('Wm2')
        data_sub['Wm2Front'] = Wm2Front
        Wm2Back = reardata['Wm2']
        data_sub['Wm2Back'] = Wm2Back
        analysis.backRatio = [x/(y+.001) for x,y in zip(reardata['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        data_sub['Back/FrontRatio'] = analysis.backRatio
        df = pd.DataFrame.from_dict(data_sub)
        df.to_csv(os.path.join("results", savefile), sep = ',',columns = ['x','y','z','rearZ','mattype','rearMat','Wm2Front','Wm2Back','Back/FrontRatio'], index = False)
    else:
        df = pd.DataFrame.from_dict(data_sub)
        df.to_csv(os.path.join("results", savefile), sep = ',', columns = ['x','y','z','mattype','Wm2'], index = False)

    print('saved: %s'%(os.path.join("results", savefile)))
    return os.path.join("results", savefile)



In [ ]:
#run through list of timestamp and generate gendaylit, combine into .oct file

#NOTE:  set the gap between the torque tube and panel here.  Gap = 0 turns torque tube shading off.
# When you make a run, results will be in the /results folder.  Rename it to either '/results_noShading', '/results_10cm',
# or '/results_30cm' after a run to be able to do the plotting in the next section.

tube_gap = 0.2  # gap between torque tube and panel.  Gap = 0 turns tube shading off.

demo.makeModule(name='2m_panel',x=1,y=module_height)
for i, time in enumerate(metdata.datetime):
    #i = 12; time = metdata.datetime[i]
    filename = str(time)[0:-12].replace('-','_').replace(' ','_')
    demo.name = filename
    demo.gendaylit(metdata,i)  # Noon, June 17th
    theta = df.surface_tilt[i];height = hub_height - 0.5* math.sin(abs(theta) * math.pi / 180) * module_height
    sceneDict = {'tilt':theta,'pitch':1.0/gcr,'height':height,'orientation':'portrait','azimuth':df.surface_azimuth[i]}  
    scene = demo.makeScene('2m_panel',sceneDict, nMods = 10, nRows = 3) #makeScene creates a .rad file with 20 modules per row, 7 rows.
    # get the .radfile name and open it
    radfile = scene.radfile; print(radfile)
    
    ##
    if tube_gap > 0:
        f = open(radfile,'a')   #a for append
        # add in a N-S torque tube, 10cm diameter, starting 10cm - 30cm distant
        # depending on tilt, needs to be offset...
        # scene uses bottom of module at x = 0 reference
        offset = tube_gap + 0.05 # 10cm offset (variable) + 5cm radius (constant)
        xtrans = 0.5*module_height*np.cos(theta*np.pi/180) / np.sin(df.surface_azimuth[i]-180) +offset*np.sin(df.tracker_theta[i]*np.pi/180)
        ztrans = hub_height-offset*math.cos(abs(theta)*math.pi / 180)
        f.writelines("\n!genrev Metal_Grey pole1 t*10 .05 32 | xform -rx -90 -t {} -5 {} -a 2 -t {} 0 0 \n".format(xtrans,ztrans,-1.0/gcr))  #20m length, 5cm radius
        #f.writelines("\n!genbox Metal_Grey pole1 0.1 0.1 10 | xform -t -.05 -.05 0 -rx -90 -t {} -5 {} -a 2 -t {} 0 0 \n".format(xtrans,ztrans,-1.0/gcr))  #20m length, 5cm radius
        f.close()
    
    
    #Use objview to inspect the new file.  this shouldn't be done interactively since it locks out the terminal.
    #run "objview materials\\ground.rad objects\\radfile.rad" from the command prompt
    #-vp 1.5 -14 0 -vd -.1 .98 .12

    ##
    octfile = demo.makeOct(demo.getfilelist())
    analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

    # create a new front and backscan
    scene.frontscan['xstart'] = scene.frontscan['xstart'] / 4
    scene.frontscan['xinc'] = scene.frontscan['xinc'] / 4
    scene.frontscan['Nx'] = scene.frontscan['Nx'] * 4
    linepts = linePtsMakeDict(scene.frontscan)
    #frontDict = irrPlotNew(octfile,linepts,demo.name+'_Front')        
    frontDict = analysis.irrPlotNew(octfile,linepts,demo.name+'_Front',accuracy='high')        

    #bottom view. 
    scene.backscan['xstart'] = scene.backscan['xstart'] / 4
    scene.backscan['xinc'] = scene.backscan['xinc'] / 4
    scene.backscan['Nx'] = scene.backscan['Nx'] * 4
    scene.backscan['zstart'] = hub_height-module_height/2*np.sin(theta*np.pi/180)-.03
    scene.backscan['zinc'] = 0.1 * module_height*np.sin(theta*np.pi/180) /4
    linepts = linePtsMakeDict(scene.backscan)
    #backDict = irrPlotNew(octfile,linepts,demo.name+'_Back')
    backDict = analysis.irrPlotNew(octfile,linepts,demo.name+'_Back',accuracy='high')
    
    ##
    #analysis.saveResults(frontDict, backDict,'irr_%s.csv'%(demo.name) )
    saveResults(frontDict, backDict,'irr_%s.csv'%(demo.name) )
    
    ##
    print('Annual %s: %0.3f - %0.3f' %(demo.name,min(analysis.backRatio), np.mean(analysis.backRatio)) )
 

In [ ]:
# read-back the values and tabulate average values for unshaded, 10cm gap and 30cm gap
# sum am and pm values separately because positive x values are always at the top half of the module
# This way will make +x and -x values symmetric.  Split AM and PM around index 7 (12pm)
import glob
import os
folder = r'10x3 36 scan v022\results_noShading'
filenames = glob.glob(os.path.join(folder,'*.csv'))


# sum across all hours
unsh_front = np.array([pd.read_csv(f, engine='python')['Wm2Front'] for f in filenames]).sum(axis = 0)
unsh_back_am = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[0:7].sum(axis = 0)
unsh_back_pm = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[7:].sum(axis = 0)
unsh_back = unsh_back_am +np.flip(unsh_back_pm,0)
unsh_back_raw = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])
# read-back 10cm data
folder =  r'10x3 36 scan v022\results_10cm'
#folder =  r'results'
filenames = glob.glob(os.path.join(folder,'*.csv'))
# sum across all hours
cm10_front = np.array([pd.read_csv(f, engine='python')['Wm2Front'] for f in filenames]).sum(axis = 0)
cm10_back_am = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[0:7].sum(axis = 0)
cm10_back_pm = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[7:].sum(axis = 0)
cm10_back = cm10_back_am + np.flip(cm10_back_pm,0)  
cm10_back_raw = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])

# read-back 20cm data
folder =  r'10x3 36 scan v022\results_20cm'
filenames = glob.glob(os.path.join(folder,'*.csv'))
# sum across all hours
cm20_front = np.array([pd.read_csv(f, engine='python')['Wm2Front'] for f in filenames]).sum(axis = 0)
cm20_back_am = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[0:7].sum(axis = 0)
cm20_back_pm = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[7:].sum(axis = 0)
cm20_back = cm20_back_am + np.flip(cm20_back_pm,0)
cm20_back_raw = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])

# read-back 30cm data
folder =  r'10x3 36 scan v022\results_30cm'
filenames = glob.glob(os.path.join(folder,'*.csv'))
# sum across all hours
cm30_front = np.array([pd.read_csv(f, engine='python')['Wm2Front'] for f in filenames]).sum(axis = 0)
cm30_back_am = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[0:7].sum(axis = 0)
cm30_back_pm = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])[7:].sum(axis = 0)
cm30_back = cm30_back_am + np.flip(cm30_back_pm,0)
cm30_back_raw = np.array([pd.read_csv(f, engine='python')['Wm2Back'] for f in filenames])


In [ ]:
print cm10_front


In [ ]:
# plot spatial loss values for 10cm and 30cm data
    
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['axes.linewidth'] = 0.2 #set the value globally

fig = plt.figure()
fig.set_size_inches(4, 2.5)
ax = fig.add_axes((0.15,0.15,0.78,0.75))
#plt.rc('font', family='sans-serif')
plt.rc('xtick',labelsize=8)
plt.rc('ytick',labelsize=8)
plt.rc('axes',labelsize=8)
plt.plot(np.linspace(-1,1,unsh_back.__len__()),(cm30_back - unsh_back)/unsh_back*100, label = '30cm gap',color = 'black')  #steelblue
plt.plot(np.linspace(-1,1,unsh_back.__len__()),(cm20_back - unsh_back)/unsh_back*100, label = '20cm gap',color = 'steelblue', linestyle = '--')  #steelblue
plt.plot(np.linspace(-1,1,unsh_back.__len__()),(cm10_back - unsh_back)/unsh_back*100, label = '10cm gap',color = 'darkorange')  #steelblue
plt.ylabel('$G_{rear}$ vs unshaded [%]')#(r'$BG_E$ [%]')
plt.xlabel('Module X position [m]')
plt.legend(fontsize = 8,frameon = False,loc='best')
plt.ylim([-19, 2])
plt.title('Torque tube shading loss',fontsize=9)
#plt.annotate('South',xy=(-10,9.5),fontsize = 8); plt.annotate('North',xy=(8,9.5),fontsize = 8)
plt.show()

In [ ]:


import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['axes.linewidth'] = 0.2 #set the value globally


#plt.rc('font', family='sans-serif')
plt.rc('xtick',labelsize=8)
plt.rc('ytick',labelsize=8)
plt.rc('axes',labelsize=8)

fig = plt.figure()
fig.set_size_inches(6, 4)
ax = fig.add_axes((0.2,0.15,0.75,0.75))
diff10 = cm10_back_raw - unsh_back_raw
for i,line in enumerate(diff10[2:-2]):
    time = str(i+7) +':00'
    xval = np.linspace(-1,1,unsh_back.__len__())
    if i > 6:
        xval = np.linspace(1,-1,unsh_back.__len__())
    if i in [3,5,7]:
        plt.plot(xval,line, label = time)  #steelblue

plt.ylabel('$G_{rear}$ vs unshaded [Wm-2]')#(r'$BG_E$ [%]')
plt.xlabel('Module X position [m]')
plt.legend(fontsize = 8,frameon = False,loc='lower left')
#plt.ylim([0, 15])
plt.title('Torque tube shading loss, 10cm gap',fontsize=9)
#plt.annotate('South',xy=(-10,9.5),fontsize = 8); plt.annotate('North',xy=(8,9.5),fontsize = 8)
plt.show()


diff30 = cm30_back_raw - unsh_back_raw
fig = plt.figure()
fig.set_size_inches(6, 4)
ax = fig.add_axes((0.2,0.15,0.75,0.75))
for i,line in enumerate(diff30[2:-2]):
    time = str(i+7) +':00'
    xval = np.linspace(-1,1,unsh_back.__len__())
    if i > 6:
        xval = np.linspace(1,-1,unsh_back.__len__())
    if i in [3,5,7]:
        plt.plot(xval,line, label = time)  #steelblue

plt.ylabel('$G_{rear}$ vs unshaded [Wm-2]')#(r'$BG_E$ [%]')
plt.xlabel('Module X position [m]')
plt.legend(fontsize = 8,frameon = False,loc='lower left')
#plt.ylim([0, 15])
plt.title('Torque tube shading loss, 30cm gap',fontsize=9)
#plt.annotate('South',xy=(-10,9.5),fontsize = 8); plt.annotate('North',xy=(8,9.5),fontsize = 8)
plt.show()


In [ ]:
# look at spatial average
print('Rear shading loss cumulative')
print((unsh_back.sum()-cm10_back.sum())/unsh_back.sum())
print((unsh_back.sum()-cm20_back.sum())/unsh_back.sum())
print((unsh_back.sum()-cm30_back.sum())/unsh_back.sum())


'''
plt.plot(np.linspace(-1,1,unsh_back.__len__()),(cm30_back - unsh_back)/unsh_back*100, label = '30cm gap',color = 'black')  #steelblue
plt.plot(np.linspace(-1,1,unsh_back.__len__()),(cm20_back - unsh_back)/unsh_back*100, label = '20cm gap',color = 'steelblue', linestyle = '--')  #steelblue
plt.plot(np.linspace(-1,1,unsh_back.__len__()),(cm10_back - unsh_back)/unsh_back*100, label = '10cm gap',color = 'darkorange')  #steelblue
'''

In [ ]:

# look at scene with rvu.  use cmd for this since it locks out the terminal.
#'rvu -vp 1.5 -14 0 -vd -.1 .98 .12  -e .01 1972_06_24_13.oct'  # look from south
#'rvu -vp 1 -14 1.5 -vd .2 .98 0  -e .01 1972_06_24_13.oct'  # look from south

In [ ]:
# create image. This can take some time 
analysis.makeImage('PVSC2018.vp')

In [ ]:
# create falsecolor. This can take some time.  Not currently working for some reason...
analysis.makeFalseColor('PVSC2018.vp')
